# Оценка эффективности каналов привлечения пользователей
***
Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения.
Данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

### Обратите внимание на следующее:
- В выгрузке только уникальные действия пользователей за каждый день;

- Можно миновать стадию установки приложения, если оно было установлено ранее;

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку.

In [1]:
# импортируем требуемые библиотеки
import numpy as np
import pandas as pd
import scipy.stats as ss

In [8]:
# загрузим данные и посмотрим на них
url_base = 'https://getfile.dokpub.com/yandex/get/'
url_kc = 'https://disk.yandex.ru/d/Ta-weYfGM5CHjg'
df = pd.read_csv(url_base + url_kc, parse_dates=[0], keep_date_col=True)
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [3]:
# посмотрим на размер датафрейма
df.shape

(2747968, 8)

In [4]:
# какие уникальные типы ивентов мы фиксируем
df.event.unique()

array(['app_start', 'choose_item', 'purchase', 'search', 'tap_basket',
       'app_install', 'register'], dtype=object)

In [7]:
# посмотрим на типы данных в датафрейме
df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

In [10]:
# сделаем столбец с номером месяца события
df['month_number'] = df.date.dt.month
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month_number
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,1
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,1
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,1
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,1
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,1


In [11]:
'''Вычислить MAU февраля'''
feb_mau = df[df.month_number == 2].device_id.nunique()
feb_mau

75032

In [12]:
'''Вычислить количество установок в январе'''
jan_inst = df[(df.month_number == 1) & (df.event == 'app_install')].device_id.count()
jan_inst

80297

In [14]:
'''Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки
в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей?
Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки
прошло не более 7 дней.'''
# Определим когорту каждого пользователя по дню установки приложения
inst_date = df[df.event == 'app_install'][['device_id', 'date']].rename(columns={'date': 'cohort'})
inst_date

,device_id,cohort
2515061,4921563,2020-01-01
2515062,1311583,2020-01-01
2515063,23006325,2020-01-01
2515064,5645652,2020-01-01
2515065,8798567,2020-01-01
...,...,...
2669653,19717968,2020-03-31
2669654,32966665,2020-03-31
2669655,6335964,2020-03-31
2669656,29155826,2020-03-31


In [15]:
# определим дату первой покупки пользователей
purchase_min_date = df[df.event == 'purchase'][['device_id', 'date']] \
                        .groupby('device_id', as_index=False) \
                        .agg({'date': 'min'}) \
                        .rename(columns={'date': 'purchase_date'})
purchase_min_date

,device_id,purchase_date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07
...,...,...
70870,35321120,2020-03-25
70871,35326839,2020-03-28
70872,35342310,2020-03-27
70873,35367797,2020-03-29


In [16]:
# объединим данные по установках и первым покупкам
cr_data = inst_date.merge(purchase_min_date, on='device_id', how='inner')
cr_data

,device_id,cohort,purchase_date
0,4921563,2020-01-01,2020-01-10
1,23006325,2020-01-01,2020-01-01
2,8423608,2020-01-01,2020-01-06
3,532710,2020-01-01,2020-01-03
4,528071,2020-01-01,2020-01-02
...,...,...,...
57453,228794,2020-03-31,2020-03-31
57454,20224705,2020-03-31,2020-03-31
57455,1434534,2020-03-31,2020-03-31
57456,271586,2020-03-31,2020-03-31


In [17]:
# добавим колонку с разницей между датой установки и покупки
cr_data['purchase_delta'] = cr_data['purchase_date'] - cr_data['cohort']
cr_data

,device_id,cohort,purchase_date,purchase_delta
0,4921563,2020-01-01,2020-01-10,9 days
1,23006325,2020-01-01,2020-01-01,0 days
2,8423608,2020-01-01,2020-01-06,5 days
3,532710,2020-01-01,2020-01-03,2 days
4,528071,2020-01-01,2020-01-02,1 days
...,...,...,...,...
57453,228794,2020-03-31,2020-03-31,0 days
57454,20224705,2020-03-31,2020-03-31,0 days
57455,1434534,2020-03-31,2020-03-31,0 days
57456,271586,2020-03-31,2020-03-31,0 days


In [18]:
# оставим только тех, у кого разница не больше 7 дней
cr_data = cr_data[cr_data.purchase_delta <= '7 days']
cr_data

,device_id,cohort,purchase_date,purchase_delta
1,23006325,2020-01-01,2020-01-01,0 days
2,8423608,2020-01-01,2020-01-06,5 days
3,532710,2020-01-01,2020-01-03,2 days
4,528071,2020-01-01,2020-01-02,1 days
5,263017,2020-01-01,2020-01-04,3 days
...,...,...,...,...
57453,228794,2020-03-31,2020-03-31,0 days
57454,20224705,2020-03-31,2020-03-31,0 days
57455,1434534,2020-03-31,2020-03-31,0 days
57456,271586,2020-03-31,2020-03-31,0 days


In [19]:
# определим количество покупок по когортам
purchase_num = cr_data.groupby('cohort', as_index=False) \
                      .agg({'device_id': 'count'}) \
                      .rename(columns={'device_id': 'purchase_num'})
purchase_num

,cohort,purchase_num
0,2020-01-01,1408
1,2020-01-02,1186
2,2020-01-03,834
3,2020-01-04,639
4,2020-01-05,587
...,...,...
86,2020-03-27,277
87,2020-03-28,209
88,2020-03-29,223
89,2020-03-30,204


In [20]:
# определим количество установок по когортам
install_num = inst_date.groupby('cohort', as_index=False) \
                       .agg({'device_id': 'count'}) \
                       .rename(columns={'device_id': 'install_num'})
install_num

,cohort,install_num
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671
...,...,...
86,2020-03-27,1199
87,2020-03-28,1091
88,2020-03-29,1117
89,2020-03-30,994


In [22]:
# объединим эти данные, для дальнейшего расчета конверсии
cr_df = purchase_num.merge(install_num, on='cohort', how='inner')
cr_df

,cohort,purchase_num,install_num
0,2020-01-01,1408,3579
1,2020-01-02,1186,3144
2,2020-01-03,834,2402
3,2020-01-04,639,1831
4,2020-01-05,587,1671
...,...,...,...
86,2020-03-27,277,1199
87,2020-03-28,209,1091
88,2020-03-29,223,1117
89,2020-03-30,204,994


In [23]:
# добавим колонку конверсии
cr_df['cr'] = (cr_df.purchase_num * 100 / cr_df.install_num).round(1)
cr_df

,cohort,purchase_num,install_num,cr
0,2020-01-01,1408,3579,39.3
1,2020-01-02,1186,3144,37.7
2,2020-01-03,834,2402,34.7
3,2020-01-04,639,1831,34.9
4,2020-01-05,587,1671,35.1
...,...,...,...,...
86,2020-03-27,277,1199,23.1
87,2020-03-28,209,1091,19.2
88,2020-03-29,223,1117,20.0
89,2020-03-30,204,994,20.5


In [24]:
# определим когорту с наибольшей конверсией
cr_df.loc[cr_df.cr.idxmax()].cohort.strftime('%d.%m.%Y')

'01.01.2020'

In [25]:
'''Укажите значение CR из предыдущего вопроса в % (округлите до 1 цифры после запятой)'''
cr_df.cr.max()

39.3

In [29]:
'''С какого платного маркетингового канала пришло больше всего новых пользователей?'''
df[(df.event == 'app_install') & (df.utm_source != '-')].utm_source.value_counts().idxmax()

'yandex-direct'

In [31]:
'''Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. 
Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. 
На каком шаге отваливается больше всего зарегистрированных пользователей?'''
# определим дату регистрации пользователей
reg_min_date = df[df.event == 'register'][['device_id', 'date']] \
                                         .rename(columns={'date': 'registration_date'})
reg_min_date

,device_id,registration_date
2669658,294193,2020-01-01
2669659,22917617,2020-01-01
2669660,15248490,2020-01-01
2669661,252062,2020-01-01
2669662,2251583,2020-01-01
...,...,...
2747963,2984778,2020-03-31
2747964,27301864,2020-03-31
2747965,1294285,2020-03-31
2747966,3010574,2020-03-31


In [32]:
# определим дату первого поиска на сайте
search_min_date = df[df.event == 'search'][['device_id', 'date']] \
                      .groupby('device_id', as_index=False) \
                      .agg({'date': 'min'}) \
                      .rename(columns={'date': 'search_date'})
search_min_date

,device_id,search_date
0,4013,2020-01-15
1,4014,2020-01-02
2,4016,2020-01-04
3,4018,2020-03-27
4,4046,2020-01-04
...,...,...
184483,35379281,2020-03-29
184484,35380796,2020-03-31
184485,35381595,2020-03-30
184486,35388218,2020-03-31


In [35]:
# объединим данные по регистрации и поиску товара пользователями
total_action = reg_min_date.merge(search_min_date, on='device_id', how='left')
total_action

,device_id,registration_date,search_date
0,294193,2020-01-01,2020-01-01
1,22917617,2020-01-01,2020-01-01
2,15248490,2020-01-01,2020-01-01
3,252062,2020-01-01,2020-01-01
4,2251583,2020-01-01,2020-01-01
...,...,...,...
78305,2984778,2020-03-31,2020-03-28
78306,27301864,2020-03-31,2020-03-31
78307,1294285,2020-03-31,2020-03-31
78308,3010574,2020-03-31,2020-03-06


In [36]:
# выберем только те данные, где регистрация перед поиском. Вычленим id пользователей
id_frame = total_action[total_action.registration_date <= total_action.search_date].device_id.to_frame()
id_frame

,device_id
0,294193
1,22917617
2,15248490
3,252062
4,2251583
...,...
78303,4294936
78304,9613546
78306,27301864
78307,1294285


In [37]:
# определим количество поисковых запросов пользователей
search_sum_action = df[df.event == 'search'][['device_id', 'date']] \
                      .groupby('device_id', as_index=False) \
                      .agg({'date': 'count'}) \
                      .rename(columns={'date': 'search_count'})
search_sum_action

,device_id,search_count
0,4013,1
1,4014,31
2,4016,4
3,4018,1
4,4046,36
...,...,...
184483,35379281,1
184484,35380796,1
184485,35381595,2
184486,35388218,1


In [38]:
# количество добавлений товара по пользователям
choose_sum_action = df[df.event == 'choose_item'][['device_id', 'date']] \
                      .groupby('device_id', as_index=False) \
                      .agg({'date': 'count'}) \
                      .rename(columns={'date': 'choose_count'})
choose_sum_action

,device_id,choose_count
0,4013,1
1,4014,21
2,4016,4
3,4018,1
4,4046,28
...,...,...
155686,35371575,1
155687,35380796,1
155688,35381595,1
155689,35388218,1


In [39]:
# количество переходов в корзину по пользователям
basket_sum_action = df[df.event == 'tap_basket'][['device_id', 'date']] \
                      .groupby('device_id', as_index=False) \
                      .agg({'date': 'count'}) \
                      .rename(columns={'date': 'basket_count'})
basket_sum_action

,device_id,basket_count
0,4013,1
1,4014,14
2,4016,2
3,4018,1
4,4046,18
...,...,...
125409,35370030,1
125410,35371575,1
125411,35380796,1
125412,35381595,1


In [40]:
# добавим к отобранным id пользователей данные по количеству действий 
total_cr = id_frame.merge(search_sum_action, on='device_id', how='left') \
                   .merge(choose_sum_action, on='device_id', how='left') \
                   .merge(basket_sum_action, on='device_id', how='left')
total_cr

,device_id,search_count,choose_count,basket_count
0,294193,1,1,1
1,22917617,19,15,9
2,15248490,88,65,45
3,252062,62,43,29
4,2251583,35,26,19
...,...,...,...,...
53223,4294936,1,1,1
53224,9613546,1,1,1
53225,27301864,1,1,1
53226,1294285,1,1,1


In [41]:
# сделаем датафрейм с суммой по всем действиям
total_sum_action = pd.DataFrame({'search_count': [total_cr.search_count.sum()], 
                                 'choose_count': [total_cr.choose_count.sum()], 
                                 'basket_count': [total_cr.basket_count.sum()]})
total_sum_action

,search_count,choose_count,basket_count
0,214762,176827,140535


In [43]:
# посчитаем конверсии в следующий этап и выберем наименьшую
total_sum_action['choose_cr'] = total_sum_action.choose_count * 100 / total_sum_action.search_count
total_sum_action['basket_cr'] = total_sum_action.basket_count * 100 / total_sum_action.choose_count
total_sum_action

,search_count,choose_count,basket_count,choose_cr,basket_cr
0,214762,176827,140535,82.336261,79.475985


In [44]:
'''Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?'''
# посчитаем количество установок по каналам
source_install = df[df.event == 'app_install'][['device_id', 'utm_source']]
source_install_num = source_install.groupby('utm_source', as_index=False) \
                                   .agg({'device_id': 'count'}) \
                                   .rename(columns={'device_id': 'install_num'})
source_install_num

,utm_source,install_num
0,-,32460
1,facebook_ads,13916
2,google_ads,26286
3,instagram_ads,20096
4,referal,9282
5,vk_ads,23189
6,yandex-direct,29368


In [50]:
# посчитаем пользователей совершивших хотя бы одну покупку по каналам
source_purchase = df[df.event == 'purchase'][['device_id', 'utm_source']].drop_duplicates()
source_purchase_num = source_purchase.groupby('utm_source', as_index=False) \
                                     .agg({'device_id': 'count'}) \
                                     .rename(columns={'device_id': 'purchase_num'})
source_purchase_num

,utm_source,purchase_num
0,-,16598
1,facebook_ads,9017
2,google_ads,11339
3,instagram_ads,10762
4,referal,6362
5,vk_ads,12364
6,yandex-direct,12028


In [52]:
# объединим данные
source_cr = source_install_num.merge(source_purchase_num, on='utm_source', how='inner')
source_cr

,utm_source,install_num,purchase_num
0,-,32460,16598
1,facebook_ads,13916,9017
2,google_ads,26286,11339
3,instagram_ads,20096,10762
4,referal,9282,6362
5,vk_ads,23189,12364
6,yandex-direct,29368,12028


In [53]:
# посчитаем конверсию
source_cr['cr'] = source_cr.purchase_num * 100 / source_cr.install_num
source_cr

,utm_source,install_num,purchase_num,cr
0,-,32460,16598,51.133703
1,facebook_ads,13916,9017,64.795918
2,google_ads,26286,11339,43.137031
3,instagram_ads,20096,10762,53.552946
4,referal,9282,6362,68.541263
5,vk_ads,23189,12364,53.318384
6,yandex-direct,29368,12028,40.956143


In [58]:
# определим канал с наименьшей конверсией
source_cr.loc[source_cr.cr.idxmin()].utm_source

'yandex-direct'

In [64]:
'''Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? 
(учитываются только первые покупки пользователей)'''
# определим дату первой покупки пользователей
date_first_purchase = inst_date.merge(purchase_min_date, on='device_id', how='inner') \
                               .rename(columns={'purchase_date': 'date'}) \
                               [['device_id', 'date']]
date_first_purchase

,device_id,date
0,4921563,2020-01-10
1,23006325,2020-01-01
2,8423608,2020-01-06
3,532710,2020-01-03
4,528071,2020-01-02
...,...,...
57453,228794,2020-03-31
57454,20224705,2020-03-31
57455,1434534,2020-03-31
57456,271586,2020-03-31


In [65]:
# добавим данные по каналам и сумме покупки
source_first_purchase = date_first_purchase.merge(df[df.purchase_sum > 0] \
                                                  [['device_id', 'date', 'purchase_sum', 'utm_source']], \
                                                 on=['device_id', 'date'], how='inner')
source_first_purchase

,device_id,date,purchase_sum,utm_source
0,4921563,2020-01-10,311.0,-
1,23006325,2020-01-01,479.5,-
2,8423608,2020-01-06,634.0,vk_ads
3,532710,2020-01-03,395.5,vk_ads
4,528071,2020-01-02,4303.5,instagram_ads
...,...,...,...,...
57453,228794,2020-03-31,473.5,facebook_ads
57454,20224705,2020-03-31,88.0,instagram_ads
57455,1434534,2020-03-31,525.5,referal
57456,271586,2020-03-31,6502.0,-


In [67]:
# сгруппируем по каналам и посмотрим медианные чеки
median_first_purchase = source_first_purchase.groupby('utm_source', as_index=False) \
                                             .agg({'purchase_sum': 'median'}) \
                                             .rename(columns={'purchase_sum': 'median_purchase'})
median_first_purchase

,utm_source,median_purchase
0,-,399.00
1,facebook_ads,389.50
2,google_ads,391.50
3,instagram_ads,392.50
4,referal,395.00
5,vk_ads,392.50
6,yandex-direct,392.75


In [68]:
# выберем канал с наибольшим медианным чеком
median_first_purchase.loc[median_first_purchase[median_first_purchase.utm_source != '-'] \
                          .median_purchase.idxmax()].utm_source

'referal'

In [69]:
'''Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Какой платный канал привлечения имеет самый высокий ROMI?'''

# посчитаем суммарный доход по каналам
source_revenue = df.groupby('utm_source', as_index=False) \
                   .agg({'purchase_sum': 'sum'})
source_revenue

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [70]:
# создадим из новых данных датафрейм
source_expense = pd.DataFrame({'utm_source': ['facebook_ads', 'google_ads', 'instagram_ads', 'vk_ads', 'yandex-direct'],
                    'expense': [8590498, 10534878, 8561626, 9553531, 10491707]})
source_expense

,utm_source,expense
0,facebook_ads,8590498
1,google_ads,10534878
2,instagram_ads,8561626
3,vk_ads,9553531
4,yandex-direct,10491707


In [73]:
# объединим таблицы
romi_calc = source_revenue.merge(source_expense, on='utm_source', how='inner')
romi_calc

,utm_source,purchase_sum,expense
0,facebook_ads,12249901.0,8590498
1,google_ads,12868276.0,10534878
2,instagram_ads,14546969.0,8561626
3,vk_ads,16389652.5,9553531
4,yandex-direct,13915368.0,10491707


In [74]:
# просчитаем ROMI
romi_calc['romi'] = ((romi_calc.purchase_sum - romi_calc.expense) * 100 / romi_calc.expense).round(1)
romi_calc

,utm_source,purchase_sum,expense,romi
0,facebook_ads,12249901.0,8590498,42.6
1,google_ads,12868276.0,10534878,22.1
2,instagram_ads,14546969.0,8561626,69.9
3,vk_ads,16389652.5,9553531,71.6
4,yandex-direct,13915368.0,10491707,32.6


In [75]:
# Выберем канал с самым высоким ROMI
romi_calc.loc[romi_calc.romi.idxmax()].utm_source

'vk_ads'

## Выводы
***
1. Когорта '01.01.2020' имеет наибольшую конверсию;
2. С Яндекс-директа пришло больше всего новых пользователей;
3. Больше всего пользователей отваливается при переходе в корзину;
4. Самый большой медианный первый чек у канала - реферальная программа;
5. При заданных исходных данных, самый высокий ROMI у VK-ads.